In [1]:
import sqlite3

import pandas as pd
from semsimian import Semsimian


from pheval_elder.prepare.core.data_processing.data_processor import DataProcessor
from pheval_elder.prepare.core.store.chromadb_manager import ChromaDBManager

db = f"/Users/ck/.data/oaklib/hp.db"
conn = sqlite3.connect(db)
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = res.fetchall()
tables



[('prefix',),
 ('rdf_list_statement',),
 ('rdf_level_summary_statistic',),
 ('anonymous_expression',),
 ('anonymous_class_expression',),
 ('anonymous_property_expression',),
 ('anonymous_individual_expression',),
 ('owl_restriction',),
 ('owl_complex_axiom',),
 ('has_oio_synonym_statement',),
 ('repair_action',),
 ('problem',),
 ('lexical_problem',),
 ('relation_graph_construct',),
 ('subgraph_query',),
 ('entailed_edge',),
 ('term_association',),
 ('statements',)]

In [2]:
print(conn)

In [5]:
df_term_association = pd.read_sql_query("SELECT * FROM entailed_edge", conn)
df_term_association.head()

,subject,predicate,object
0,UBERON:0001772,rdfs:subClassOf,UBERON:0001772
1,HP:0031972,rdfs:subClassOf,HP:0031972
2,HP:0000679,rdfs:subClassOf,HP:0000679
3,HP:0008245,rdfs:subClassOf,HP:0000821
4,HP:0000821,rdfs:subClassOf,HP:0000821


In [9]:
db_manager = ChromaDBManager(path="/Users/ck/Monarch/elder/emb_data/models/large3", collection_name="large3_lrd_hpo_embeddings")
dataprocessor = DataProcessor(db_manager)

hps = list(dataprocessor.hp_embeddings.keys())[:1000]


In [7]:
hps[:4]

['HP:0033468', 'HP:0033469', 'HP:0033470', 'HP:0033471']

In [10]:

predicates = ["rdfs:subClassOf"]
rss = Semsimian(spo=None, predicates=predicates, pairwise_similarity_attributes=None, resource_path=db)
all_x_all = rss.all_by_all_pairwise_similarity(subject_terms=set(hps), object_terms=set(hps), minimum_jaccard_threshold=0, minimum_resnik_threshold=0)


In [15]:
rows_list = []
for term1_key, values in all_x_all.items():
    for term2_key, result in values.items():
        jaccard, resnik, phenodigm, _, _ = result
        row_dict = {"subject": term1_key, "object": term2_key, "jaccard": jaccard, "aic": resnik, "phenodigm": phenodigm}
        rows_list.append(row_dict)

new_df = pd.DataFrame(rows_list)

In [16]:
new_df.head()